# Chunking text with a maximum number of tokens and a percentage of overlapping

This code demonstrate how to use Langchain with tiktoken technique to chunk content with a maximum number of tokens (i.e. 512 or 1024) and with a percentage of overlapping between chunks.

This notebook is based on this article from Langchain: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/split_by_token/#tiktoken

The output is the chunks created from a long text to be vectorized and indexed 

## Prerequisites

We used Python 3.12.5, [Visual Studio Code with the Python extension](https://code.visualstudio.com/docs/python/python-tutorial), and the [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) to test this example.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
! pip install --upgrade --quiet langchain-text-splitters tiktoken

## Import packages and create Langchain's text splitter client

In [1]:
import os
import sys

sys.path.append('../..')
from rag_utils import load_files, token_len

# CONSTANTS
MAX_CHUNK_TOKEN_SIZE = 512 # Max number of tokens for chunking
CHUNK_OVERLAPPING=0.25 # 25% of overlapping between chunks

## Chunking with Langchain and tiktoken

In [2]:
from rag_utils import chunk_with_max_tokens

In [ ]:
# Sample: chunking a document in markdown format
text = """
Descripción: Ayuda Elypse
===
||
| - |
| Apartados que cambian Descripción del cambio Fecha entrada en vigor Fecha actualización del documento Lanzamiento promoción 23/06/2020 24/06/2020 |
| |
| 1 1 ¿En qué consiste la promoción? |
| |
| La Liga y Mediapro han llegado a un acuerdo con los operadores para que puedan ofrecer el fútbol gratis (Canal La Liga TV) en las residencias de ancianos Telefónica participa de este acuerdo dando a las residencias un producto con los mismos contenidos que el de BAR pero a un precio mucho más competitivo Al cliente le hablamos de un producto para residencias a un precio muy competitivo y con el canal LaLiga TV gratis, no de una promoción sobre el producto BAR A quién aplica: Residencias de ancianos (incluidas en el códidgo CNAE 8731 -asistencia en establecimientos residenciales para personas mayores-) que llamen para solicitar el fútbol gratuito en sus residencias (zonas comunes y cafetería) También para residencias que tengan el producto BAR instalado en la cafetería o zona común y soliciten el fútbol gratis Cuando llame al 1004 o acuda a una tienda el responsable de una residencia de ancianos para solicitar el producto, derivar siempre la demanda a su comercial o al 900 26 22 62 El PVP del producto es de 112 al mes IVA incluido (pese a que el fútbol -LIGA TV- es gratis, el producto tiene otros contenidos que sí tienen un coste) hasta el 31 de agosto de 2020 |
| |
| Asegura siempre |
| |
| Informa al cliente de que esta promoción será válida solo para residencias de ancianos; aquellas incluidas en el código CNAE 8731 Se harán comprobaciones antes y después de instalar el equipo |
| |
| Contratación y disfrute |
| |
| Fechas de contratación Fechas o periodo de disfrute Desde 23/06/2020 Hasta 31/08/2020 Hasta el 31 de agosto |
| |
| |
| 1 2 ¿A quién va dirigida? |
| |
| Residencias de ancianos Solo disponible en tecnologí­a 1P SAT Segmento del cliente: Autónomos Empresas |
| |
| |
| 1 3 Incompatibilidades |
| |
| No aplica |
| |
| |
| 1 4 Canales de Venta |
| |
| Otros Instaladores DTS y Digitex |
| |
| |
| 1 5 Comunicación |
| |
| Campañas en Medios Otros soportes de comunicación N/A N/A |
| |
| |
| 1 6 Mantenimiento de la promoción |
| |
| No aplica |
| |
| |
| 1 7 Compromiso de la promoción y movimientos con penalización |
| |
| No aplica |
| |
| |
| 1 8 Hitos de la promoción (detallados) |
| |
| Fecha Descripción 23/06/2020 Lanzamiento promoción Residencias de ancianos 112 al mes hasta el 31 de agosto de 2020 |
| |
. 
Argumentación: Ayuda Elypse
===
# 2 Argumentación
||
| - |
| CLAIM COMERCIAL: Para clientes que soliciten el nuevo producto para residencias de ancianos Movistar es consciente de la situación que se ha generando y lo impactadas que se han visto las residencias de ancianos Por eso quiere ofrecer un producto de difusión pública con las mejores condiciones posibles Para que nuestros mayores puedan disfrutar del fútbol |
| |
. 
Precios y Factura: ERROR DOC INTEL. 
Tramitación: Ayuda Elypse
===
# 4 Tramitación

4 1 Tramitación TELCO
No aplica, ya que estos productos no están disponibles en Telco

4 2 Tramitación no TELCO
No aplica
"""

print(f'total tokens: {token_len(text)}')

# Chunking with Langchain
chunks = chunk_with_max_tokens(text, max_tokens=MAX_CHUNK_TOKEN_SIZE, overlap=CHUNK_OVERLAPPING)

i=1
for chunk in chunks:
    print(f'* Chunk {i}, num. tokens: {token_len(chunk)},\nchunk: [{chunk}]')
    print('---------------------------------------------------------------------')
    i+=1
print(f'total number of chunks: {i-1}')

## Chunk every txt file in the input directory and write them in the output directory

In [ ]:
# Chunk markdown files and write the chunks as files in the output directory
input_dir = '../../data_out/markdown_files'
output_dir = '../../data_out/chunk_files'
os.makedirs(output_dir,exist_ok=True)
markdown_contents = load_files(input_dir, '.txt')

for i, markdown_content in enumerate(markdown_contents):
    print(f"[{i + 1}]: title: {markdown_content['title']}")
    print(f"\t content: [{markdown_content['content']}]")

    chunks = chunk_with_max_tokens(markdown_content['content'], max_tokens=MAX_CHUNK_TOKEN_SIZE, overlap=CHUNK_OVERLAPPING)
    # Write every chunk in a file in the output directory
    for j, chunk in enumerate(chunks):
        print(f'* Chunk {j + 1}, num. tokens: {token_len(chunk)},\nchunk: [{chunk}]')
        chunk_filename = markdown_content['title'].replace(".txt", f"_{j}.txt")
        file_path = os.path.join(output_dir, chunk_filename)
        print(f"\tWritting file [{file_path}]")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(chunk)
